In [1]:
import torch
import torch.nn as nn
import math

In [28]:
class FireflyAlgorithm:
    def __init__(self, func, dim, n_fireflies=20, alpha=0.5, beta0=1.0, gamma=1.0, lb=-10, ub=10, max_iter=100):
        self.func = func
        self.dim = dim
        self.n_fireflies = n_fireflies
        self.alpha = alpha
        self.beta0 = beta0
        self.gamma = gamma
        self.lb = lb
        self.ub = ub
        self.max_iter = max_iter
        self.best_solution = None
        self.best_fitness = float('inf')

    def optimize(self):
        # Initialize firefly positions
        fireflies = self.lb + (self.ub - self.lb) * torch.rand((self.n_fireflies, self.dim))
        intensities = torch.zeros(self.n_fireflies)

        # Main optimization loop
        for t in range(self.max_iter):
            # Evaluate intensities
            for i in range(self.n_fireflies):
                intensities[i] = self.func(fireflies[i])

            # Update best solution
            min_idx = torch.argmin(intensities)
            if intensities[min_idx] < self.best_fitness:
                self.best_solution = fireflies[min_idx].clone()
                self.best_fitness = intensities[min_idx]

            # Update firefly positions
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if intensities[j] < intensities[i]:
                        r = torch.norm(fireflies[i] - fireflies[j])
                        beta = self.beta0 * math.exp(-self.gamma * r**2)
                        fireflies[i] += beta * (fireflies[j] - fireflies[i]) + self.alpha * (torch.rand(self.dim) - 0.5)

                        # Ensure new position is within bounds
                        fireflies[i] = torch.clamp(fireflies[i], self.lb, self.ub)

        return self.best_solution, self.best_fitness

In [29]:
# Define a linear regression model
model = nn.Sequential(
    nn.Linear(3, 1),
)

In [30]:
# Define the mean squared error loss
criterion = nn.MSELoss()

In [31]:

# Define the function to be optimized (MSE loss)
def func(weights):
    model[0].weight.data = torch.tensor(weights.clone().detach().reshape(1, -1))
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    return loss.item()

In [32]:
# Generate some dummy data
inputs = torch.randn(100, 3)
targets = torch.mm(inputs, torch.Tensor([[1, 2, 3]]).t())

In [39]:
# Instantiate the FireflyAlgorithm optimizer
fa = FireflyAlgorithm(func, dim=3, max_iter=1000)

In [40]:
# Optimize the model
best_solution, best_fitness = fa.optimize()

/tmp/ipykernel_38626/2641362432.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model[0].weight.data = torch.tensor(weights.clone().detach().reshape(1, -1))


In [41]:

# Set the best solution as the weights of the linear regression model
model[0].weight.data = torch.tensor(best_solution.reshape(1, -1))

/tmp/ipykernel_38626/115733666.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model[0].weight.data = torch.tensor(best_solution.reshape(1, -1))


In [42]:
model

Sequential(
  (0): Linear(in_features=3, out_features=1, bias=True)
)

In [43]:
model(torch.Tensor([1, 1, 1]))

tensor([5.6015], grad_fn=<ViewBackward0>)

In [44]:
model[0].weight.data

tensor([[1.0773, 2.0265, 3.0097]])